In [1]:
# Import libraries for data consolidation and further processing
import pandas as pd
from pathlib import Path 
import numpy as np
import os
import glob
import re

In [2]:
# SET PROJECT ROOT
PROJECT_ROOT = Path().resolve().parent

# SET DATA DIRECTORY
DATA_DIR = PROJECT_ROOT / "data"

# SET CLEANED DIRECTORY
CLEANED_DIR = DATA_DIR / "cleaned"

# SET COMBINED DIRECTORY
COMBINED_DIR = DATA_DIR / "combined_data"

In [3]:
# Import all cleaned data files
team_roster_data_path = CLEANED_DIR / "team_roster_data.csv"
point_distribution_data_path = CLEANED_DIR / "point_distribution_data.csv"
efficiency_tempo_data_path = CLEANED_DIR / "efficiency_tempo_data.csv"
team_season_data_path = CLEANED_DIR / "team_season_data.csv"

# Assign KENPOM DataFrames
TEAMROSTERDATA_CLEANED_df = pd.read_csv(team_roster_data_path)
POINT_DISTRIBUTION_CLEANED_df = pd.read_csv(point_distribution_data_path)
EFFICIENCY_TEMPO_CLEANED_df = pd.read_csv(efficiency_tempo_data_path)

# ASSIGN KAGGLE DATAFRAME
TEAM_SEASON_CLEANED_df = pd.read_csv(team_season_data_path)

In [4]:
# Combine TEAMROSTERDATA_CLEANED_df, POINT_DISTRIBUTION_CLEANED_df, and EFFICIENCY_TEMPO_CLEANED_df into a single DataFrame
combined_kenpom_df = pd.merge(TEAMROSTERDATA_CLEANED_df, POINT_DISTRIBUTION_CLEANED_df, on=["season", "team_name"], how="outer")
combined_kenpom_df = pd.merge(combined_kenpom_df, EFFICIENCY_TEMPO_CLEANED_df, on=["season", "team_name"], how="outer")

In [5]:
# Find shape of all kenPom DataFrames and print them
team_roster_shape = TEAMROSTERDATA_CLEANED_df.shape
point_distribution_shape = POINT_DISTRIBUTION_CLEANED_df.shape
efficiency_tempo_shape = EFFICIENCY_TEMPO_CLEANED_df.shape

# Print shapes of all kenPom DataFrames
print(f"TEAMROSTERDATA_CLEANED_df shape: {team_roster_shape}")
print(f"POINT_DISTRIBUTION_CLEANED_df shape: {point_distribution_shape}")
print(f"EFFICIENCY_TEMPO_CLEANED_df shape: {efficiency_tempo_shape}")

TEAMROSTERDATA_CLEANED_df shape: (2157, 50)
POINT_DISTRIBUTION_CLEANED_df shape: (2157, 14)
EFFICIENCY_TEMPO_CLEANED_df shape: (2153, 19)


In [6]:
combined_kenpom_df.shape

(2158, 79)

In [7]:
# Export combined__kenpom_df to CSV
combined_kenpom_df.to_csv(COMBINED_DIR / "combined_kenpom_data.csv", index=False)
#print(f"Exported combined__kenpom_df to {COMBINED_DIR / 'combined_kenpom_data.csv'}")

In [8]:
# Load Team spellings dataframe
KAGGLE_DIR = DATA_DIR / "kaggleData"
TEAM_SPELLINGS_df = pd.read_csv(KAGGLE_DIR / "MTeamSpellings.csv")

In [58]:
# Set all team names in combined_kenpom_df to lowercase
combined_kenpom_df['team_name'] = combined_kenpom_df['team_name'].str.lower()
# Assign teamID to cleaned kenpom_df based off TEAM_SPELLINGS_df's TeamID
KENPOM_DF = combined_kenpom_df.merge(TEAM_SPELLINGS_df, left_on="team_name", right_on="TeamNameSpelling", how="left")
KENPOM_DF.rename(columns={"TeamID": "team_id"}, inplace=True)
KENPOM_DF

,season,team_name,avgHeightOnCourt_MinutesWeighted,avgHeightOnCourt_MinutesWeighted_Rank,avgCenterHeight,avgCenterHeight_Rank,avgPowerForwardHeight,avgPowerForwardHeight_Rank,avgSmallForwardHeight,avgSmallForwardHeight_Rank,...,adjusted_offensive_efficiency_rating,adjusted_offensive_efficiency_rating_rank,raw_offensive_efficiency_rating,adjusted_offensive_rating_rank,adjusted_defensive_efficiency_rating,adjusted_defensive_efficiency_rating_rank,raw_defensive_efficiency_rating,raw_defensive_efficiency_rating_rank,TeamNameSpelling,team_id
0,2019,abilene christian,75.89,299.0,-0.87,271.0,-0.71,260.0,-0.93,295.0,...,103.4,196.0,105.8,110.0,102.0,121.0,97.6,48.0,abilene christian,1101
1,2019,air force,77.15,126.0,-2.01,340.0,0.19,145.0,0.33,113.0,...,100.6,252.0,98.8,275.0,107.7,242.0,105.7,242.0,air force,1102
2,2019,akron,76.31,249.0,0.27,128.0,-0.96,287.0,0.20,125.0,...,100.0,259.0,99.4,264.0,94.1,22.0,94.0,11.0,akron,1103
3,2019,alabama,77.88,39.0,-0.01,161.0,0.69,97.0,1.08,65.0,...,108.2,103.0,102.5,185.0,97.5,58.0,102.3,154.0,alabama,1104
4,2019,alabama a&m,75.77,313.0,-1.47,322.0,-0.55,240.0,-0.04,174.0,...,87.0,350.0,88.8,347.0,109.1,264.0,104.9,214.0,alabama a&m,1105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207,2025,xavier,77.31,165.0,0.07,182.0,0.91,63.0,0.25,143.0,...,116.2,47.0,111.8,64.0,99.0,48.0,102.5,81.0,xavier,1462
2208,2025,yale,77.79,94.0,-0.79,250.0,-0.59,257.0,0.90,62.0,...,115.2,59.0,117.1,14.0,104.2,115.0,102.7,84.0,yale,1463
2209,2025,yale,77.79,94.0,-0.79,250.0,-0.59,257.0,0.90,62.0,...,115.2,59.0,117.1,14.0,104.2,115.0,102.7,84.0,yale,1463
2210,2025,youngstown st.,76.84,238.0,2.00,19.0,-1.09,305.0,0.16,155.0,...,102.4,249.0,104.6,215.0,105.3,137.0,102.9,91.0,youngstown st.,1464


In [88]:
# Show IsNa values in KENPOM_DF
na_counts = KENPOM_DF.isna().sum()
print("NA values in KENPOM_DF:")
for col, count in na_counts.items():
    if count > 0:
        print(f"- {col}: {count} NA values")
        

NA values in KENPOM_DF:
- avgHeightOnCourt_MinutesWeighted: 1 NA values
- avgHeightOnCourt_MinutesWeighted_Rank: 1 NA values
- avgCenterHeight: 1 NA values
- avgCenterHeight_Rank: 1 NA values
- avgPowerForwardHeight: 1 NA values
- avgPowerForwardHeight_Rank: 1 NA values
- avgSmallForwardHeight: 1 NA values
- avgSmallForwardHeight_Rank: 1 NA values
- avgShootingGuardHeight: 1 NA values
- avgShootingGuardHeight_Rank: 1 NA values
- avgPointGuardHeight: 1 NA values
- avgPointGuardHeight_Rank: 1 NA values
- heightEfficiencyRating: 1 NA values
- heightEfficiencyRating_Rank: 1 NA values
- avgYearsOfExperience: 1 NA values
- avgYearsOfExperience_Rank: 1 NA values
- avgBenchPoints: 1 NA values
- avgBenchPoints_Rank: 1 NA values
- avgCenterPoints: 1 NA values
- avgCenterPoints_Rank: 1 NA values
- avgPowerForwardPoints: 1 NA values
- avgPowerForwardPoints_Rank: 1 NA values
- avgSmallForwardPoints: 1 NA values
- avgSmallForwardPoints_Rank: 1 NA values
- avgShootingGuardPoints: 1 NA values
- avgSho

In [89]:
# Remove all rows with NA values in KENPOM_DF
KENPOM_DF = KENPOM_DF.dropna()

In [126]:
# Merge KENPOM_DF with TEAM_SEASON_CLEANED_df to get season data on team ID
CONSOLIDATED_DF = KENPOM_DF.merge(TEAM_SEASON_CLEANED_df, left_on=["season", "team_id"], right_on=["season", "team_id"], how="left")

In [127]:
# Show consolidated DataFrame information
CONSOLIDATED_DF.info()

# Show all object columns in CONSOLIDATED_DF
object_columns = CONSOLIDATED_DF.select_dtypes(include=['object']).columns
print("Object columns in CONSOLIDATED_DF:")
for col in object_columns:
    print(f"- {col}")


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2206 entries, 0 to 2205
Columns: 102 entries, season to avg_pts_ag
dtypes: float64(95), int64(2), object(5)
memory usage: 1.7+ MB
Object columns in CONSOLIDATED_DF:
- team_name_x
- conference_x
- TeamNameSpelling
- team_name_y
- conference_y


In [128]:
# Print is NA Count in CONSOLIDATED_DF
na_counts_consolidated = CONSOLIDATED_DF.isna().sum()
print("NA values in CONSOLIDATED_DF:")
for col, count in na_counts_consolidated.items():
    if count > 0:
        print(f"- {col}: {count}")
    else:
        print(f"- {col}: NONE")


NA values in CONSOLIDATED_DF:
- season: NONE
- team_name_x: NONE
- avgHeightOnCourt_MinutesWeighted: NONE
- avgHeightOnCourt_MinutesWeighted_Rank: NONE
- avgCenterHeight: NONE
- avgCenterHeight_Rank: NONE
- avgPowerForwardHeight: NONE
- avgPowerForwardHeight_Rank: NONE
- avgSmallForwardHeight: NONE
- avgSmallForwardHeight_Rank: NONE
- avgShootingGuardHeight: NONE
- avgShootingGuardHeight_Rank: NONE
- avgPointGuardHeight: NONE
- avgPointGuardHeight_Rank: NONE
- heightEfficiencyRating: NONE
- heightEfficiencyRating_Rank: NONE
- avgYearsOfExperience: NONE
- avgYearsOfExperience_Rank: NONE
- avgBenchPoints: NONE
- avgBenchPoints_Rank: NONE
- avgCenterPoints: NONE
- avgCenterPoints_Rank: NONE
- avgPowerForwardPoints: NONE
- avgPowerForwardPoints_Rank: NONE
- avgSmallForwardPoints: NONE
- avgSmallForwardPoints_Rank: NONE
- avgShootingGuardPoints: NONE
- avgShootingGuardPoints_Rank: NONE
- avgPointGuardPoints: NONE
- avgPointGuardPoints_Rank: NONE
- avgCenterOffensiveRebounds: NONE
- avgCente

In [129]:
# Print each column in CONSOLIDATED_DF
for col in CONSOLIDATED_DF.columns:
    print(f"{col}")

season
team_name_x
avgHeightOnCourt_MinutesWeighted
avgHeightOnCourt_MinutesWeighted_Rank
avgCenterHeight
avgCenterHeight_Rank
avgPowerForwardHeight
avgPowerForwardHeight_Rank
avgSmallForwardHeight
avgSmallForwardHeight_Rank
avgShootingGuardHeight
avgShootingGuardHeight_Rank
avgPointGuardHeight
avgPointGuardHeight_Rank
heightEfficiencyRating
heightEfficiencyRating_Rank
avgYearsOfExperience
avgYearsOfExperience_Rank
avgBenchPoints
avgBenchPoints_Rank
avgCenterPoints
avgCenterPoints_Rank
avgPowerForwardPoints
avgPowerForwardPoints_Rank
avgSmallForwardPoints
avgSmallForwardPoints_Rank
avgShootingGuardPoints
avgShootingGuardPoints_Rank
avgPointGuardPoints
avgPointGuardPoints_Rank
avgCenterOffensiveRebounds
avgCenterOffensiveRebounds_Rank
avgPowerForwardOffensiveRebounds
avgPowerForwardOffensiveRebounds_Rank
avgSmallForwardOffensiveRebounds
avgSmallForwardOffensiveRebounds_Rank
avgShootingGuardOffensiveRebounds
avgShootingGuardOffensiveRebounds_Rank
avgPointGuardOffensiveRebounds
avgPointGu

In [130]:
# Drop  'team_name_x' and 'team_name_y' columns from CONSOLIDATED_DF
CONSOLIDATED_DF = CONSOLIDATED_DF.drop(columns=['team_name_x', 'team_name_y'], errors='ignore')

# Rename 'TeamNameSpelling' to team_name in CONSOLIDATED_DF
CONSOLIDATED_DF.rename(columns={"TeamNameSpelling": "team_name"}, inplace=True)

# Drop 'conference_x' column from CONSOLIDATED_DF
CONSOLIDATED_DF = CONSOLIDATED_DF.drop(columns=['conference_x'], errors='ignore')

# Rename 'conference_y' to 'conference' in CONSOLIDATED_DF
CONSOLIDATED_DF.rename(columns={"conference_y": "conference"}, inplace=True)

In [131]:
# Print column count
print(f"Total columns: {len(CONSOLIDATED_DF.columns)}")

# Print shape of CONSOLIDATED_DF
print(f"Shape of CONSOLIDATED_DF: {CONSOLIDATED_DF.shape}")

# Reorder columns in CONSOLIDATED_DF
column_order = [
    'season', 'team_id', 'team_name', 'conference',
    'games_played', 'wins', 'win_pct',
    'avg_fgm', 'avg_fga', 'avg_fgm3', 'avg_fga3',
    'avg_ftm', 'avg_fta', 'avg_orb',
    'avg_drb', 'avg_ast', 'avg_to', 'avg_stl',
    'avg_blk', 'avg_pf', 'avg_pts_for', 'avg_pts_ag',
    'coach_change',
    'avgHeightOnCourt_MinutesWeighted', 'avgHeightOnCourt_MinutesWeighted_Rank',
    'avgCenterHeight', 'avgCenterHeight_Rank',
    'avgPowerForwardHeight', 'avgPowerForwardHeight_Rank',
    'avgSmallForwardHeight', 'avgSmallForwardHeight_Rank',
    'avgShootingGuardHeight', 'avgShootingGuardHeight_Rank',
    'avgPointGuardHeight', 'avgPointGuardHeight_Rank',
    'heightEfficiencyRating', 'heightEfficiencyRating_Rank',
    'avgYearsOfExperience', 'avgYearsOfExperience_Rank',
    'avgBenchPoints', 'avgBenchPoints_Rank',
    'avgCenterPoints', 'avgCenterPoints_Rank',
    'avgPowerForwardPoints', 'avgPowerForwardPoints_Rank',
    'avgSmallForwardPoints', 'avgSmallForwardPoints_Rank',
    'avgShootingGuardPoints', 'avgShootingGuardPoints_Rank',
    'avgPointGuardPoints', 'avgPointGuardPoints_Rank',
    'avgCenterOffensiveRebounds','avgCenterOffensiveRebounds_Rank',
    'avgPowerForwardOffensiveRebounds','avgPowerForwardOffensiveRebounds_Rank',
    'avgSmallForwardOffensiveRebounds','avgSmallForwardOffensiveRebounds_Rank',
    'avgShootingGuardOffensiveRebounds','avgShootingGuardOffensiveRebounds_Rank',
    'avgPointGuardOffensiveRebounds','avgPointGuardOffensiveRebounds_Rank',
    'avgCenterDefensiveRebounds','avgCenterDefensiveRebounds_Rank',
    'avgPowerForwardDefensiveRebounds','avgPowerForwardDefensiveRebounds_Rank',
    'avgSmallForwardDefensiveRebounds','avgSmallForwardDefensiveRebounds_Rank',
    'avgShootingGuardDefensiveRebounds','avgShootingGuardDefensiveRebounds_Rank',
    'avgPointGuardDefensiveRebounds','avgPointGuardDefensiveRebounds_Rank',
    'ft_point_share_offense','ft_point_share_offense_rank',
    'two_point_share_offense','two_point_share_offense_rank',
    'three_point_share_offense','three_point_share_offense_rank',
    'ft_point_share_defense','ft_point_share_defense_rank',
    'two_point_share_defense','two_point_share_defense_rank',
    'three_point_share_defense','three_point_share_defense_rank',
    'adjusted_tempo_rating','adjusted_tempo__rating_rank',
    'raw_tempo_rating','raw_tempo_rating_rank',
    'offensive_avg_possession_length','offensive_avg_possession_length_rank',
    'defensive_avg_possession_length','defensive_avg_possession_length_rank',
    'adjusted_offensive_efficiency_rating','adjusted_offensive_efficiency_rating_rank',
    'raw_offensive_efficiency_rating','adjusted_offensive_rating_rank',
    'adjusted_defensive_efficiency_rating','adjusted_defensive_efficiency_rating_rank',
    'raw_defensive_efficiency_rating','raw_defensive_efficiency_rating_rank'
]
CONSOLIDATED_DF = CONSOLIDATED_DF[column_order]


Total columns: 99
Shape of CONSOLIDATED_DF: (2206, 99)


In [132]:
# SET EDA DIRECTORY
EDA_DIR = DATA_DIR / "eda_data"

# export CONSOLIDATED_DF to CSV
CONSOLIDATED_DF.to_csv(EDA_DIR / "edaData_AllSeasons.csv", index=False)
#print(f"Exported CONSOLIDATED_DF to {COMBINED_DIR / 'combinedData_AllSeasons.csv'}")

# export where season is 2019 to CSV
CONSOLIDATED_DF_2019 = CONSOLIDATED_DF[CONSOLIDATED_DF['season'] == 2019]
CONSOLIDATED_DF_2019.to_csv(EDA_DIR / "edaData_2019.csv", index=False)
#print(f"Exported CONSOLIDATED_DF_2019 to {COMBINED_DIR / 'combined_2019.csv'}")

# export where season is 2021 to CSV
CONSOLIDATED_DF_2021 = CONSOLIDATED_DF[CONSOLIDATED_DF['season'] == 2021]
CONSOLIDATED_DF_2021.to_csv(EDA_DIR / "edaData_2021.csv", index=False)
#print(f"Exported CONSOLIDATED_DF_2021 to {COMBINED_DIR / 'combined_2021.csv'}")

# export where season is 2022 to CSV
CONSOLIDATED_DF_2022 = CONSOLIDATED_DF[CONSOLIDATED_DF['season'] == 2022]
CONSOLIDATED_DF_2022.to_csv(EDA_DIR / "edaData_2022.csv", index=False)
#print(f"Exported CONSOLIDATED_DF_2022 to {COMBINED_DIR / 'combined_2022.csv'}")

# export where season is 2023 to CSV
CONSOLIDATED_DF_2023 = CONSOLIDATED_DF[CONSOLIDATED_DF['season'] == 2023]
CONSOLIDATED_DF_2023.to_csv(EDA_DIR / "edaData_2023.csv", index=False)
#print(f"Exported CONSOLIDATED_DF_2023 to {COMBINED_DIR / 'combined_2023.csv'}")

# export where season is 2024 to CSV
CONSOLIDATED_DF_2024 = CONSOLIDATED_DF[CONSOLIDATED_DF['season'] == 2024]
CONSOLIDATED_DF_2024.to_csv(EDA_DIR / "edaData_2024.csv", index=False)
#print(f"Exported CONSOLIDATED_DF_2024 to {COMBINED_DIR / 'combined_2024.csv'}")

# export where season is 2025 to CSV
CONSOLIDATED_DF_2025 = CONSOLIDATED_DF[CONSOLIDATED_DF['season'] == 2025]
CONSOLIDATED_DF_2025.to_csv(EDA_DIR / "edaData_2025.csv", index=False)
#print(f"Exported CONSOLIDATED_DF_2025 to {COMBINED_DIR / 'combined_2025.csv'}") 

print("Data consolidation complete. All data exported to EDA directory.")

Data consolidation complete. All data exported to EDA directory.
